In [134]:
import requests
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
import nltk
import re
import string 
import regex


In [62]:
!pip install pymorphy2
!pip install pymystem3

  Using cached https://files.pythonhosted.org/packages/a3/33/fff9675c68b5f6c63ec8c6e6ff57827dda28a1fa5b2c2d727dffff92dd47/pymorphy2-0.8-py2.py3-none-any.whl


In [3]:
import pymorphy2
import pymystem3

# Задание 1

В regex появилась возможность longest match, которая позволяет использовать условия содержащие подстроки самих себя. В re в таких случаях находились только самые короткие совпадения и никогда строки, включающие эти более короткие строки.


In [232]:
re.search(r'те|тем|теми', 'тем')

<re.Match object; span=(0, 2), match='те'>

In [229]:
regex.search(r'те|тем|теми', 'тем', flags=regex.POSIX)

<regex.Match object; span=(0, 3), match='тем'>

# Задание 2

Загружаем текст с http://gramota.ru/class/coach/punct/45_177

In [4]:
page = requests.get("http://gramota.ru/class/coach/punct/45_177")
soup = BeautifulSoup(page.content, 'html.parser')
task = soup.find(id="taskarea")
text = task.get_text()


Приводим текст к нижнему регистру, удаляем все знаки пунктуации, токенизируем:

In [7]:
text = text.lower()
text = re.sub(r'<>[\n][\xa0]', ' ', text )
clear_text = re.sub(r'[–!?\.:;\)\(,_…]', '', text)
tokens = clear_text.split()

In [221]:
clear_text[0:400]

'\nобособление обстоятельств \nобстоятельство  второстепенный член предложения который обозначает признак действия или другого признака обстоятельства поясняют сказуемые или другие члены предложения при синтаксическом разборе предложения обстоятельства подчеркиваются пунктирной линией тире точка тире обстоятельства необходимо выделять запятыми в трех случаях рассмотрим каждый из них по очереди \nпервы'

In [15]:
tokens[:10]

['обособление',
 'обстоятельств',
 'обстоятельство',
 'второстепенный',
 'член',
 'предложения',
 'который',
 'обозначает',
 'признак',
 'действия']

Делаем стемминг:

In [102]:
stemmer = SnowballStemmer('russian')
stems = [[token, stemmer.stem(token), morph.parse(token)[0].normal_form] for token in tokens]
print(stems)

[['обособление', 'обособлен', 'обособление'], ['обстоятельств', 'обстоятельств', 'обстоятельство'], ['обстоятельство', 'обстоятельств', 'обстоятельство'], ['второстепенный', 'второстепен', 'второстепенный'], ['член', 'член', 'член'], ['предложения', 'предложен', 'предложение'], ['который', 'котор', 'который'], ['обозначает', 'обознача', 'обозначать'], ['признак', 'признак', 'признак'], ['действия', 'действ', 'действие'], ['или', 'ил', 'или'], ['другого', 'друг', 'другой'], ['признака', 'признак', 'признак'], ['обстоятельства', 'обстоятельств', 'обстоятельство'], ['поясняют', 'поясня', 'пояснять'], ['сказуемые', 'сказуем', 'сказуемое'], ['или', 'ил', 'или'], ['другие', 'друг', 'другой'], ['члены', 'член', 'член'], ['предложения', 'предложен', 'предложение'], ['при', 'при', 'при'], ['синтаксическом', 'синтаксическ', 'синтаксический'], ['разборе', 'разбор', 'разбор'], ['предложения', 'предложен', 'предложение'], ['обстоятельства', 'обстоятельств', 'обстоятельство'], ['подчеркиваются', 'по

Ищем ошибки:
    
    1) два разных слова ошибочно свелись к одинаковой основе ('как' и 'каким' свелись к основе 'как', 'этого' и 'это' свелись к основе 'эт', 'так' и 'такие' свелись к основе 'так', 'не' и 'нее' свелись к 'не')

In [122]:
res = []
for i in range(len(stems)):
    for j in range(1,len(stems)):
        if stems[i][1]==stems[j][1] and stems[i][0]!=stems[j][0] and stems[i][2]!=stems[j][2] and i!=j:
            res.append((tuple(stems[i]), tuple(stems[j])))
set(res)

{(('вид', 'вид', 'вид'), ('видит', 'вид', 'видеть')),
 (('видит', 'вид', 'видеть'), ('вид', 'вид', 'вид')),
 (('видит', 'вид', 'видеть'), ('видов', 'вид', 'вид')),
 (('видов', 'вид', 'вид'), ('видит', 'вид', 'видеть')),
 (('второй', 'втор', 'два'), ('втором', 'втор', 'второе')),
 (('втором', 'втор', 'второе'), ('второй', 'втор', 'два')),
 (('выделяется', 'выделя', 'выделяться'), ('выделять', 'выделя', 'выделять')),
 (('выделять', 'выделя', 'выделять'), ('выделяется', 'выделя', 'выделяться')),
 (('выделять', 'выделя', 'выделять'), ('выделяются', 'выделя', 'выделяться')),
 (('выделяются', 'выделя', 'выделяться'), ('выделять', 'выделя', 'выделять')),
 (('выражением', 'выражен', 'выражение'),
  ('выраженные', 'выражен', 'выразить')),
 (('выраженные', 'выражен', 'выразить'),
  ('выражением', 'выражен', 'выражение')),
 (('готовиться', 'готов', 'готовиться'), ('готовы', 'готов', 'готовый')),
 (('готовы', 'готов', 'готовый'), ('готовиться', 'готов', 'готовиться')),
 (('долго', 'долг', 'долго')

    
    2) две словоформы одного слова после стемминга отличаются друг от друга
    

In [123]:
res =[]
for i in range(len(stems)):
    for j in range(1,len(stems)):
        if stems[i][2]==stems[j][2] and stems[i][1]!= stems[j][1] and i!=j:
            res.append((tuple(stems[i]), tuple(stems[j])))
set(res)

{(('будет', 'будет', 'быть'), ('были', 'был', 'быть')),
 (('будет', 'будет', 'быть'), ('было', 'был', 'быть')),
 (('будет', 'будет', 'быть'), ('быть', 'быт', 'быть')),
 (('были', 'был', 'быть'), ('будет', 'будет', 'быть')),
 (('были', 'был', 'быть'), ('быть', 'быт', 'быть')),
 (('было', 'был', 'быть'), ('будет', 'будет', 'быть')),
 (('было', 'был', 'быть'), ('быть', 'быт', 'быть')),
 (('быть', 'быт', 'быть'), ('будет', 'будет', 'быть')),
 (('быть', 'быт', 'быть'), ('были', 'был', 'быть')),
 (('быть', 'быт', 'быть'), ('было', 'был', 'быть')),
 (('в', 'в', 'в'), ('во', 'во', 'в')),
 (('видел', 'видел', 'видеть'), ('видит', 'вид', 'видеть')),
 (('видит', 'вид', 'видеть'), ('видел', 'видел', 'видеть')),
 (('во', 'во', 'в'), ('в', 'в', 'в')),
 (('времени', 'времен', 'время'), ('время', 'врем', 'время')),
 (('время', 'врем', 'время'), ('времени', 'времен', 'время')),
 (('все', 'все', 'весь'), ('всем', 'всем', 'весь')),
 (('всем', 'всем', 'весь'), ('все', 'все', 'весь')),
 (('второй', 'втор',

    
    3) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов)

In [12]:
res = set()
for i in range(len(stems)):
    if stems[i][0]==stems[i][1] and len(stems[i][0])>4:
        res.add(stems[i])
print(res)

{('перед', 'перед'), ('через', 'через'), ('выйдет', 'выйдет'), ('грузовик', 'грузовик'), ('будет', 'будет'), ('воннегут', 'воннегут'), ('имеет', 'имеет'), ('вынырнул', 'вынырнул'), ('вопрос', 'вопрос'), ('заснул', 'заснул'), ('сотрудник', 'сотрудник'), ('рак-отшельник', 'рак-отшельник'), ('видел', 'видел'), ('хотел', 'хотел'), ('например', 'например'), ('могут', 'могут'), ('подряд', 'подряд'), ('сумел', 'сумел'), ('оборот', 'оборот'), ('дворник', 'дворник'), ('салтыков-щедрин', 'салтыков-щедрин'), ('обстоятельств', 'обстоятельств'), ('город', 'город'), ('возраст', 'возраст'), ('признак', 'признак'), ('может', 'может'), ('сидел', 'сидел'), ('мороз', 'мороз'), ('вдруг', 'вдруг'), ('взмахнув', 'взмахнув'), ('пушкин', 'пушкин'), ('назад', 'назад'), ('пошел', 'пошел'), ('встает', 'встает')}


# Задание 3

In [66]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariasamsonova/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [171]:
import nltk
from nltk import corpus
from nltk.corpus import stopwords

stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

"Около, могли, теми, собой, часто" - эти слова дополняют имеющиеся в списке стоп-слов парадигмы предлогов, модальных глаголов, местоимений указательных и возвратных и наречий частотности. 

# Задание 4

Загружаем текст с https://zona.media/news/2019/09/11/zhukov-expertise и очищаем его.

In [124]:
page = requests.get("https://zona.media/news/2019/09/11/zhukov-expertise")
soup = BeautifulSoup(page.content, 'html.parser')
text_zona = [text.get_text() for text in list(soup.find_all("section", {'class':'mz-publish__text'}))][0]

In [125]:
text_zona

'\u200b«Новая газета» опубликовала сокращенный вариант заключения экспертов Александра Коршикова и Анны Осокиной, которые анализировали видео из блога студента Высшей школы экономики Егора Жукова, обвиняемого в призывах к\xa0экстремизму в интернете (часть 2 статьи 280 УК). Эксперты нашли экстремизм в четырех видео на YouTube-канале Жукова. По их мнению, в видео «отчетливо прослеживается мотив политической ненависти или вражды к действующей власти». В заключении говорится, что Жуков приписывает действующей власти «враждебные действия», называет ее врагами, источником зла и вреда. Кроме того, студент призывает с этой властью бороться. «Автор описывает катастрофическое положение населения в России, как следствие целенаправленной политики действующей власти <...>. Основным идеалом, которого следует достичь, является "смена власти в России" для того, чтобы "Россия стала свободной"», — цитирует издание экспертов.В одном из видео, в котором Жуков размышлял об итогах митинга 7 октября 2017 год

In [129]:
text_zona = text_zona.replace('\u200b', ' ')
text_zona = text_zona.replace('\xa0', ' ')

In [131]:
text_zona

' «Новая газета» опубликовала сокращенный вариант заключения экспертов Александра Коршикова и Анны Осокиной, которые анализировали видео из блога студента Высшей школы экономики Егора Жукова, обвиняемого в призывах к экстремизму в интернете (часть 2 статьи 280 УК). Эксперты нашли экстремизм в четырех видео на YouTube-канале Жукова. По их мнению, в видео «отчетливо прослеживается мотив политической ненависти или вражды к действующей власти». В заключении говорится, что Жуков приписывает действующей власти «враждебные действия», называет ее врагами, источником зла и вреда. Кроме того, студент призывает с этой властью бороться. «Автор описывает катастрофическое положение населения в России, как следствие целенаправленной политики действующей власти <...>. Основным идеалом, которого следует достичь, является "смена власти в России" для того, чтобы "Россия стала свободной"», — цитирует издание экспертов.В одном из видео, в котором Жуков размышлял об итогах митинга 7 октября 2017 года, экспе

Разбиваем текст на предложения и токенизируем:

In [135]:
sents = nltk.sent_tokenize(text_zona, 'russian')

In [138]:
tokens_zona = [sent.split() for sent  in sents]

In [139]:
tokens_zona

[['«Новая',
  'газета»',
  'опубликовала',
  'сокращенный',
  'вариант',
  'заключения',
  'экспертов',
  'Александра',
  'Коршикова',
  'и',
  'Анны',
  'Осокиной,',
  'которые',
  'анализировали',
  'видео',
  'из',
  'блога',
  'студента',
  'Высшей',
  'школы',
  'экономики',
  'Егора',
  'Жукова,',
  'обвиняемого',
  'в',
  'призывах',
  'к',
  'экстремизму',
  'в',
  'интернете',
  '(часть',
  '2',
  'статьи',
  '280',
  'УК).'],
 ['Эксперты',
  'нашли',
  'экстремизм',
  'в',
  'четырех',
  'видео',
  'на',
  'YouTube-канале',
  'Жукова.'],
 ['По',
  'их',
  'мнению,',
  'в',
  'видео',
  '«отчетливо',
  'прослеживается',
  'мотив',
  'политической',
  'ненависти',
  'или',
  'вражды',
  'к',
  'действующей',
  'власти».'],
 ['В',
  'заключении',
  'говорится,',
  'что',
  'Жуков',
  'приписывает',
  'действующей',
  'власти',
  '«враждебные',
  'действия»,',
  'называет',
  'ее',
  'врагами,',
  'источником',
  'зла',
  'и',
  'вреда.'],
 ['Кроме',
  'того,',
  'студент',
  'пр

In [140]:
string.punctuation
string.punctuation += '«»—…“”'

In [145]:
tokens_zona = re.findall('\w+', text_zona)

In [203]:
tokens_zona

['Новая',
 'газета',
 'опубликовала',
 'сокращенный',
 'вариант',
 'заключения',
 'экспертов',
 'Александра',
 'Коршикова',
 'и',
 'Анны',
 'Осокиной',
 'которые',
 'анализировали',
 'видео',
 'из',
 'блога',
 'студента',
 'Высшей',
 'школы',
 'экономики',
 'Егора',
 'Жукова',
 'обвиняемого',
 'в',
 'призывах',
 'к',
 'экстремизму',
 'в',
 'интернете',
 'часть',
 '2',
 'статьи',
 '280',
 'УК',
 'Эксперты',
 'нашли',
 'экстремизм',
 'в',
 'четырех',
 'видео',
 'на',
 'YouTube',
 'канале',
 'Жукова',
 'По',
 'их',
 'мнению',
 'в',
 'видео',
 'отчетливо',
 'прослеживается',
 'мотив',
 'политической',
 'ненависти',
 'или',
 'вражды',
 'к',
 'действующей',
 'власти',
 'В',
 'заключении',
 'говорится',
 'что',
 'Жуков',
 'приписывает',
 'действующей',
 'власти',
 'враждебные',
 'действия',
 'называет',
 'ее',
 'врагами',
 'источником',
 'зла',
 'и',
 'вреда',
 'Кроме',
 'того',
 'студент',
 'призывает',
 'с',
 'этой',
 'властью',
 'бороться',
 'Автор',
 'описывает',
 'катастрофическое',
 'по

Лемматизируем с помощью pymorphy2:

In [204]:
morph = pymorphy2.MorphAnalyzer()
lemmas_pymorphy = []
for token in tokens_zona:
    s =[]
    s.append(morph.parse(token)[0].word)
    s.append(morph.parse(token)[0].normal_form)
    lemmas_pymorphy.append(s)    

In [205]:
lemmas_pymorphy

[['новая', 'новый'],
 ['газета', 'газета'],
 ['опубликовала', 'опубликовать'],
 ['сокращённый', 'сократить'],
 ['вариант', 'вариант'],
 ['заключения', 'заключение'],
 ['экспертов', 'эксперт'],
 ['александра', 'александр'],
 ['коршикова', 'коршикова'],
 ['и', 'и'],
 ['анны', 'анна'],
 ['осокиной', 'осокин'],
 ['которые', 'который'],
 ['анализировали', 'анализировать'],
 ['видео', 'видео'],
 ['из', 'из'],
 ['блога', 'блог'],
 ['студента', 'студент'],
 ['высшей', 'высокий'],
 ['школы', 'школа'],
 ['экономики', 'экономика'],
 ['егора', 'егор'],
 ['жукова', 'жуков'],
 ['обвиняемого', 'обвинять'],
 ['в', 'в'],
 ['призывах', 'призыв'],
 ['к', 'к'],
 ['экстремизму', 'экстремизм'],
 ['в', 'в'],
 ['интернете', 'интернет'],
 ['часть', 'часть'],
 ['2', '2'],
 ['статьи', 'статья'],
 ['280', '280'],
 ['ук', 'ук'],
 ['эксперты', 'эксперт'],
 ['нашли', 'найти'],
 ['экстремизм', 'экстремизм'],
 ['в', 'в'],
 ['четырёх', 'четыре'],
 ['видео', 'видео'],
 ['на', 'на'],
 ['youtube', 'youtube'],
 ['канале', 

Лемматизируем текст, разбитый на предложения:

In [58]:
mystem = pymystem3.Mystem()


Installing mystem to /Users/dariasamsonova/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


['\u200b«',
 'новый',
 ' ',
 'газета',
 '» ',
 'опубликовывать',
 ' ',
 'сокращать',
 ' ',
 'вариант']

In [206]:
lemmas_mystem_sent = []
for sent in sents:
    lem_sent=[]
    words_analized = mystem.analyze(sent)
    for word in range(len(words_analized)):
        if words_analized[word].get('analysis'):
            lem_sent.append((words_analized[word]['text'], words_analized[word]['analysis'][0]['lex']))
    lemmas_mystem_sent.append(lem_sent)
print(lemmas_mystem_sent)

[[('Новая', 'новый'), ('газета', 'газета'), ('опубликовала', 'опубликовывать'), ('сокращенный', 'сокращать'), ('вариант', 'вариант'), ('заключения', 'заключение'), ('экспертов', 'эксперт'), ('Александра', 'александр'), ('Коршикова', 'коршиков'), ('и', 'и'), ('Анны', 'анна'), ('Осокиной', 'осокина'), ('которые', 'который'), ('анализировали', 'анализировать'), ('видео', 'видео'), ('из', 'из'), ('блога', 'блог'), ('студента', 'студент'), ('Высшей', 'высокий'), ('школы', 'школа'), ('экономики', 'экономика'), ('Егора', 'егор'), ('Жукова', 'жуков'), ('обвиняемого', 'обвиняемый'), ('в', 'в'), ('призывах', 'призыв'), ('к', 'к'), ('экстремизму', 'экстремизм'), ('в', 'в'), ('интернете', 'интернет'), ('часть', 'часть'), ('статьи', 'статья'), ('УК', 'ук')], [('Эксперты', 'эксперт'), ('нашли', 'находить'), ('экстремизм', 'экстремизм'), ('в', 'в'), ('четырех', 'четыре'), ('видео', 'видео'), ('на', 'на'), ('канале', 'канал'), ('Жукова', 'жуков')], [('По', 'по'), ('их', 'их'), ('мнению', 'мнение'), ('

Лемматизируем "сырой" текст:

In [207]:
words_analized = mystem.analyze(text_zona)

In [208]:
lemmas_mystem = []
for word in range(len(words_analized)):
    if words_analized[word].get('analysis'):
        lemmas_mystem.append((words_analized[word]['text'], words_analized[word]['analysis'][0]['lex']))
print(lemmas_mystem)

[('Новая', 'новый'), ('газета', 'газета'), ('опубликовала', 'опубликовывать'), ('сокращенный', 'сокращать'), ('вариант', 'вариант'), ('заключения', 'заключение'), ('экспертов', 'эксперт'), ('Александра', 'александр'), ('Коршикова', 'коршиков'), ('и', 'и'), ('Анны', 'анна'), ('Осокиной', 'осокина'), ('которые', 'который'), ('анализировали', 'анализировать'), ('видео', 'видео'), ('из', 'из'), ('блога', 'блог'), ('студента', 'студент'), ('Высшей', 'высокий'), ('школы', 'школа'), ('экономики', 'экономика'), ('Егора', 'егор'), ('Жукова', 'жуков'), ('обвиняемого', 'обвиняемый'), ('в', 'в'), ('призывах', 'призыв'), ('к', 'к'), ('экстремизму', 'экстремизм'), ('в', 'в'), ('интернете', 'интернет'), ('часть', 'часть'), ('статьи', 'статья'), ('УК', 'ук'), ('Эксперты', 'эксперт'), ('нашли', 'находить'), ('экстремизм', 'экстремизм'), ('в', 'в'), ('четырех', 'четыре'), ('видео', 'видео'), ('на', 'на'), ('канале', 'канал'), ('Жукова', 'жуков'), ('По', 'по'), ('их', 'их'), ('мнению', 'мнение'), ('в', '

# Вопросы: 

    1) Что лучше разбивает на предложения (mystem или sent_tokenize)?

В pymystem3 не реализованна так явно как в NLTK функция разбивки на предложения и это уже делает его значительно менее удобным. Если немного подлатать код обертки pymystem3, то можно заставить его делать это, но проще воспользоваться sent_tokenize или воспользоваться чьей-то умной регуляркой, они работают с достаточно хорошей точностью.




    2) Что лучше токенизирует (mystem или ваша регулярка)? Если mystem, то можно ли улучшить регулярное выражение так, чтобы качество было выше?

Mystem токенизирует лучше, например, он не разделяет составные слова, написанные через дефис, такие как из-за, кому-то и тд. Однако, Mystem не удаляет пунктуацию и из-за этого появляются сложности при работе с проанализированным текстом, можно давать на вход текст без пунктуации, но тогда придется писать регулярное выражение, которое удаляло бы только пунктуацию предложения и осталяло дефисы, а в этом случае такое регулярное выражение совсем несложно модифицируется в токеннизатор (если мы убрали все знаки препинания, то разделить по пробелам - совсем несложно) и тогда уже не будет уступать в качестве mystem.



    3) Что лучше для лемматизации mystem или pymorphy?

In [210]:
for word_p in range(len(lemmas_pymorphy)):
    for word_m in range(len(lemmas_mystem)):
        if lemmas_pymorphy[word_p][1]!=lemmas_mystem[word_m][1] and lemmas_pymorphy[word_p][0]==lemmas_mystem[word_m][0]:
            print(lemmas_pymorphy[word_p], lemmas_mystem[word_m])
            
        

['опубликовала', 'опубликовать'] ('опубликовала', 'опубликовывать')
['обвиняемого', 'обвинять'] ('обвиняемого', 'обвиняемый')
['нашли', 'найти'] ('нашли', 'находить')
['их', 'они'] ('их', 'их')
['действующей', 'действующий'] ('действующей', 'действовать')
['действующей', 'действующий'] ('действующей', 'действовать')
['того', 'тот'] ('того', 'то')
['того', 'тот'] ('того', 'то')
['действующей', 'действующий'] ('действующей', 'действовать')
['достичь', 'достигнуть'] ('достичь', 'достигать')
['того', 'тот'] ('того', 'то')
['того', 'тот'] ('того', 'то')
['стала', 'стать'] ('стала', 'становиться')
['об', 'о'] ('об', 'об')
['есть', 'есть'] ('есть', 'быть')
['этого', 'это'] ('этого', 'этот')
['отправили', 'отправить'] ('отправили', 'отправлять')
['закрыли', 'закрыть'] ('закрыли', 'закрывать')
['предъявили', 'предъявить'] ('предъявили', 'предъявлять')
['новое', 'новое'] ('новое', 'новый')


Говорить о том, какой лемматизатор работает лучше довольно сложно, потому что однозначных примеров немного: 

In [ ]:
Слова, с разбором которых pymorphy справился лучше:
['того', 'тот'] ('того', 'то')
['об', 'о'] ('об', 'об')

In [ ]:
Слова, с разбором которых mystem справился лучше:
['нашли', 'найти'] ('нашли', 'находить')
['есть', 'есть'] ('есть', 'быть')
['этого', 'это'] ('этого', 'этот')
['новое', 'новое'] ('новое', 'новый')

mystem лучше работает на формах слов значимых частей речи, образованных супплетивизмом, в то время как pymorphy лучше справляется со служебными частями речи (хотя у mystem тоже есть один пример более удачной лемматизации таких слов ['этого', 'это'] ('этого', 'этот')). 

При этом можно заметить больше примеров, когда мы не можем дать однозначного ответа на вопрос какой разбор более правильный:

In [ ]:
1)  ['действующей', 'действующий'] ('действующей', 'действовать')
    ['обвиняемого', 'обвинять'] ('обвиняемого', 'обвиняемый')   

Здесь мы видим примеры лемматизации причастий. Сложность заключается в том, что при разборе причастий в некоторых подходах их начальной формой принято считать форму причастия едиенственного числа мужского рода именительного падежа с сохранением залога и времени, в то время как в других подходах причастие - это строго глагольная форма, поэтому ее начальной формой должен быть инфинитив. Решение о том, какой лемматизатор выполнил работу лучше зависит от конкретной задачи и от степени обобщения, которая нам необходима. 



In [ ]:
2)  ['их', 'они'] ('их', 'их')

Здесь правильность приведения к начальной форме будет зависеть от контекста, потому что "их" - это омонимичная форма, которая входит в парадигмы слов "они" (их - мн ч, Р.п/В.п) и "их" (все падежи притяжательного местоимения "их"), снятие этой омонимии, насколько я знаю, происходит примерно одинаково в стандартном виде: у pymorphy просто хранится самый встречаемый вариант и он один, у mystem хранятся все варинаты, ранжированные по частотности, но по умолчанию он также выбирает самый частостный, поэтому в страндартном виде они решают. эту задачу, в прицнипе, одинаково и все будет зависеть от контекста (совпадет ли он с самым частотным вариантом из корпусов, на которых обучались лемматизаторы), но в теории то, что у mystem хранятся и другие варианты снятия омонимии - это хорошо и может быть использованно в обработке текстов.



In [ ]:
3)  ['отправили', 'отправить'] ('отправили', 'отправлять')
    ['закрыли', 'закрыть'] ('закрыли', 'закрывать')
    ['предъявили', 'предъявить'] ('предъявили', 'предъявлять')
    ['опубликовала', 'опубликовать'] ('опубликовала', 'опубликовывать')
    ['стала', 'стать'] ('стала', 'становиться')

Здесь та же проблема, что и с причастиями: нам нужно решить до какой степени мы хотим обобщать слова и хотим ли различать глаголы по виду, если мы считаем что один глагол совершенного и несовершенного вида - это одно слово, тогда правильный разбор у mystem, в противном случае - у pymorphy. Иногда сохранение вида глагола может быть важным, напрмер, если в дальнейшем мы хотим работать с семантикой, то вид глагола может сильно влиять на значение слова в целом. 

# Вывод:
Все лемматизаторы хороши по-своему, все зависит от задачи и наших потребностей :)